In [1]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install ipcalc
!{sys.executable} -m pip install graphviz

# import all necessary packages
import matplotlib.pyplot as plt # plotting
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import ipcalc

In [2]:
# Data cleaning and enhancing

# read data
nRows = 10000
data = pd.read_csv('C:/Users/Sandi/Downloads/ip-network-traffic-flows-labeled-with-87-apps/Dataset-Unicauca-Version2-87Atts.csv', sep= ',', header= 0, nrows = nRows)

# remove the columns which have no unique elements
data = data[[col for col in data if data[col].nunique() > 1]]

# Convert the columns of non numerical type to numerical type
data['Flow.ID_factor'] = data['Flow.ID'].factorize()[0]
data['Flow.Duration_factor'] = data['Flow.Duration'].factorize()[0]
data['ProtocolName_factor'] = data['ProtocolName'].factorize()[0]
data['Timestamp_factor'] = data['Timestamp'].factorize()[0]
for i in range(1,32):
    sip=[]
    dip=[]
    print("Calculating all network address for /"+str(i))
    for j in range(0,nRows):
        SourceIP = data['Source.IP'][j]+"/"+str(i)
        DestinationIP = data['Destination.IP'][j]+"/"+str(i)
        Sadd = ipcalc.Network(SourceIP)
        Dadd = ipcalc.Network(DestinationIP)
        sip.append(str(Sadd.network()))
        dip.append(str(Dadd.network()))
    data['SourceIP_'+str(i)] = pd.Series(sip)
    data['SourceIP_'+str(i)+'_factor'] = data['SourceIP_'+str(i)].factorize()[0] 
    data['DestinationIP_'+str(i)] = pd.Series(dip)
    data['DestinationIP_'+str(i)+'_factor'] = data['DestinationIP_'+str(i)].factorize()[0] 

In [3]:
data.columns.values

In [4]:
# Reindex the dataframe such that 1st column is target column
x = ['L7Protocol', 'Flow.ID_factor', 'DestinationIP_1_factor', 'DestinationIP_10_factor' ,'DestinationIP_11_factor' ,'DestinationIP_12_factor' ,
     'DestinationIP_13_factor' ,'DestinationIP_14_factor' ,'DestinationIP_15_factor' ,'DestinationIP_16_factor' ,'DestinationIP_17_factor' ,'DestinationIP_18_factor' ,
     'DestinationIP_19_factor' ,'DestinationIP_2_factor' ,'DestinationIP_20_factor' ,'DestinationIP_21_factor' ,'DestinationIP_22_factor' ,'DestinationIP_23_factor' ,
     'DestinationIP_24_factor' ,'DestinationIP_25_factor' ,'DestinationIP_26_factor' ,'DestinationIP_27_factor' ,'DestinationIP_28_factor' ,'DestinationIP_29_factor' ,
     'DestinationIP_3_factor' ,'DestinationIP_30_factor' ,'DestinationIP_31_factor' ,'DestinationIP_4_factor' ,'DestinationIP_5_factor' ,'DestinationIP_6_factor' ,
     'DestinationIP_7_factor' ,'DestinationIP_8_factor' ,'DestinationIP_9_factor' ,'SourceIP_10_factor' ,'SourceIP_11_factor' ,'SourceIP_12_factor' ,
     'SourceIP_13_factor' ,'SourceIP_14_factor' ,'SourceIP_15_factor' ,'SourceIP_16_factor' ,'SourceIP_17_factor' ,'SourceIP_18_factor' ,'SourceIP_19_factor' ,
     'SourceIP_2_factor' ,'SourceIP_20_factor' ,'SourceIP_21_factor' ,'SourceIP_22_factor' ,'SourceIP_23_factor' ,'SourceIP_24_factor' ,'SourceIP_25_factor' ,
     'SourceIP_26_factor' ,'SourceIP_27_factor' ,'SourceIP_28_factor' ,'SourceIP_29_factor' ,'SourceIP_3_factor' ,'SourceIP_30_factor' ,'SourceIP_31_factor' ,
     'SourceIP_4_factor' ,'SourceIP_5_factor' ,'SourceIP_6_factor' ,'SourceIP_7_factor' ,'SourceIP_8_factor' ,'SourceIP_9_factor' ,'SourceIP_1_factor'  'Source.Port',
     'Destination.Port', 'Timestamp_factor', 'Flow.Duration_factor','Total.Fwd.Packets', 'Total.Backward.Packets', 'Total.Length.of.Fwd.Packets', 
     'Total.Length.of.Bwd.Packets', 'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min', 'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max', 
     'Bwd.Packet.Length.Min', 'Bwd.Packet.Length.Mean','Bwd.Packet.Length.Std', 'Flow.Bytes.s', 'Flow.Packets.s','Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
     'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std',  'Bwd.IAT.Max', 
     'Bwd.IAT.Min', 'Fwd.PSH.Flags', 'Fwd.Header.Length', 'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s','Min.Packet.Length', 'Max.Packet.Length', 
     'Packet.Length.Mean', 'Packet.Length.Std', 'Packet.Length.Variance', 'FIN.Flag.Count','SYN.Flag.Count', 'RST.Flag.Count', 'PSH.Flag.Count', 'ACK.Flag.Count', 
     'URG.Flag.Count', 'ECE.Flag.Count', 'Down.Up.Ratio','Average.Packet.Size', 'Avg.Fwd.Segment.Size', 'Avg.Bwd.Segment.Size', 'Fwd.Header.Length.1', 
     'Subflow.Fwd.Packets', 'Subflow.Fwd.Bytes','Subflow.Bwd.Packets', 'Subflow.Bwd.Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 
     'min_seg_size_forward','Active.Mean', 'Active.Max', 'Active.Min', 'Idle.Mean', 'Idle.Max','Idle.Min']
'''
x = ['ProtocolName_factor', 'Flow.ID_factor', 'SourceIP_Octet1_factor','SourceIP_Octet2_factor','SourceIP_Octet3_factor','SourceIP_Octet4_factor', 'Source.Port', 'DestinationIP_Octet1_factor','DestinationIP_Octet2_factor','DestinationIP_Octet3_factor','DestinationIP_Octet4_factor','Destination.Port', 'Timestamp_factor', 'Flow.Duration_factor','Total.Fwd.Packets', 'Total.Backward.Packets', 'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets', 'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min', 'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max', 'Bwd.Packet.Length.Min', 'Bwd.Packet.Length.Mean','Bwd.Packet.Length.Std', 'Flow.Bytes.s', 'Flow.Packets.s',
'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max', 'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std',  'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags', 'Fwd.Header.Length', 'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s','Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean', 'Packet.Length.Std', 'Packet.Length.Variance', 'FIN.Flag.Count','SYN.Flag.Count', 'RST.Flag.Count', 'PSH.Flag.Count', 'ACK.Flag.Count', 'URG.Flag.Count', 'ECE.Flag.Count', 'Down.Up.Ratio','Average.Packet.Size', 'Avg.Fwd.Segment.Size', 'Avg.Bwd.Segment.Size', 'Fwd.Header.Length.1', 'Subflow.Fwd.Packets', 'Subflow.Fwd.Bytes','Subflow.Bwd.Packets', 'Subflow.Bwd.Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward','Active.Mean', 'Active.Max', 'Active.Min', 'Idle.Mean', 'Idle.Max',
'Idle.Min', 'L7Protocol']

x = ['L7Protocol', 'Flow.ID_factor', 'SourceIP_Octet1_factor','SourceIP_Octet2_factor','SourceIP_Octet3_factor','SourceIP_Octet4_factor', 'Source.Port', 'DestinationIP_Octet1_factor','DestinationIP_Octet2_factor','DestinationIP_Octet3_factor','DestinationIP_Octet4_factor','Destination.Port']
'''
X=data.reindex(columns=x[1:])
print(X.head())
X =  X[[col for col in X if X[col].nunique() > 1]]

# Preprocess the data to normalize the dataset
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
X = normalize(X)
Y = data['L7Protocol']

#Check correlation between the target column and other columns
z = X
z_colnames = z.columns.values.tolist()
z_colnames.append('L7Protocol')
z = z.reindex(columns=z_colnames)
z['L7Protocol'] = Y
corr_matrix = abs(z.corr(method='pearson'))
y=corr_matrix['L7Protocol'].sort_values(ascending=False)
print('Correlation between column X and column \'L7Protocol\'\n',y)
print(X.head())

In [5]:
# May drop the columns with low correlation
'''Column_drop = ['Average.Packet.Size','Flow.IAT.Mean','SYN.Flag.Count','Fwd.PSH.Flags','Idle.Max','Idle.Mean','Idle.Min','Flow.ID_factor','Bwd.IAT.Total',
               'Fwd.Packet.Length.Min','Min.Packet.Length','Fwd.Packet.Length.Max','Fwd.Packet.Length.Std','Avg.Fwd.Segment.Size','Fwd.Packet.Length.Mean',
               'Flow.IAT.Min','Fwd.IAT.Min','Subflow.Fwd.Bytes','Total.Length.of.Fwd.Packets','Bwd.IAT.Max','Active.Mean','Active.Min','Bwd.IAT.Min',
               'Fwd.Header.Length.1','Fwd.Header.Length','Active.Max','RST.Flag.Count','ECE.Flag.Count','FIN.Flag.Count',
               'Bwd.IAT.Mean','Subflow.Fwd.Packets','Total.Fwd.Packets','Bwd.Packet.Length.Min','Bwd.IAT.Std','Bwd.Packets.s',
               'Timestamp_factor','Init_Win_bytes_backward','act_data_pkt_fwd']
X=X.drop(columns=Column_drop)
'''
# Split the datase into training and testing dataset
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X,Y, data.index, test_size=0.2, random_state=0)
print('Data size of training set:',len(X_train))
print('Data size of testing set:',len(X_test))

In [6]:
from sklearn.svm import LinearSVC
#Linear SVC
lsvc = LinearSVC()
y_pred_LSVC = lsvc.fit(X_train, y_train).predict(X_test)
print("Accuracy of SVC: ", metrics.accuracy_score(y_test,y_pred_LSVC)*100)

#KNN
knn = KNeighborsClassifier()
y_pred_knn = knn.fit(X_train, y_train).predict(X_test)
print("Accuracy of KNN:",metrics.accuracy_score(y_test, y_pred_knn)*100)


#DTC
from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 0)
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 0)
y_pred_DTC_gini = clf_gini.fit(X_train, y_train).predict(X_test)
y_pred_DTC_entropy = clf_entropy.fit(X_train, y_train).predict(X_test)
print("Accuracy of DTC (gini): ", metrics.accuracy_score(y_test,y_pred_DTC_gini)*100)
print("Accuracy of DTC (entropy):", metrics.accuracy_score(y_test,y_pred_DTC_entropy)*100)

#RFC
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=2000, random_state=0)
y_pred_rfc = rfc.fit(X_train, y_train).predict(X_test)
print("Accuracy of Random Forest Classifier:",metrics.accuracy_score(y_test, y_pred_rfc)*100)

In [7]:
# DTC gini decision tree image:
plt.rcParams['figure.figsize'] = [20,20]
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
graph = Source(tree.export_graphviz(clf_gini.fit(X_train, y_train), out_file=None, filled = True))
display(SVG(graph.pipe(format='svg')))

In [8]:
# Saving the model

import joblib
model = rfc.fit(X_train, y_train)
filename = 'finalized_model.sav'
joblib.dump(model, filename)